In [3]:
import tensorflow as tf

In [4]:
node1 = tf.constant(3.0, tf.float32)
node2 = tf.constant(4.0)
print(node1, node2)

Tensor("Const:0", shape=(), dtype=float32) Tensor("Const_1:0", shape=(), dtype=float32)


In [5]:
sess = tf.Session()
print(sess.run([node1, node2]))

[3.0, 4.0]


In [6]:
node3 = tf.add(node1, node2)
print(node3)
print(sess.run(node3))

Tensor("Add:0", shape=(), dtype=float32)
7.0


In [7]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = tf.add(a, b)

In [8]:
print(sess.run(adder_node, {a:3, b:4.5}))
print(sess.run(adder_node, {a:[1, 2], b:[2, 3]}))

7.5
[ 3.  5.]


In [9]:
add_and_triple = adder_node * 3
print(sess.run(add_and_triple, {a:[1, 2], b:[1, 2]}))

[  6.  12.]


In [12]:
a = tf.Variable([.3], tf.float32)
b = tf.Variable([-.3], tf.float32)
x = tf.placeholder(tf.float32)
linear_model = a * x + b
init = tf.global_variables_initializer()
sess.run(init)
print(sess.run(linear_model, {x:[0, 1, 2, 3, 4]}))

[-0.30000001  0.          0.30000001  0.60000002  0.90000004]


In [13]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
print(sess.run(loss, {x:[1, 2, 3, 4], y:[0, -1, -2, -3]}))

23.66


In [17]:
fix_a = tf.assign(a, [-1.])
fix_b = tf.assign(b, [1.])
sess.run([fix_a, fix_b])
print(sess.run(loss, {x:[1, 2, 3, 4], y:[0, -1, -2, -3]}))

0.0


In [18]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)
sess.run(init)
for i in range(1000):
    sess.run(train, {x:[1, 2, 3, 4], y:[0, -1, -2, -3]})
print(sess.run([a, b]))

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


In [21]:
print(sess.run([loss], {x:[1, 2, 3, 4], y:[0, -1, -2, -3]}))

[5.6999738e-11]


In [24]:
import numpy as np
features = [tf.contrib.layers.real_valued_column("x", dimension = 1)]
estimator = tf.contrib.learn.LinearRegressor(feature_columns = features)
x = np.array([1., 2., 3., 4.])
y = np.array([0., -1., -2., -3.])
input_fn = tf.contrib.learn.io.numpy_input_fn({"x":x}, y, batch_size = 4, num_epochs = 1000)
estimator.fit(input_fn = input_fn, steps = 1000)
estimator.evaluate(input_fn = input_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_keep_checkpoint_max': 5, '_save_summary_steps': 100, '_is_chief': True, '_num_ps_replicas': 0, '_evaluation_master': '', '_save_checkpoints_secs': 600, '_environment': 'local', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_master': '', '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f2585f9aa58>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_tf_random_seed': None}
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmp8b0uv6mw/m

{'global_step': 1000, 'loss': 8.2993661e-11}

In [35]:
def model(features, labels, mode):
    A = tf.get_variable("A", [1], dtype = tf.float64)
    b = tf.get_variable("b", [1], dtype = tf.float64)
    y = A * features['x'] + b
    loss = tf.reduce_sum(tf.square(y - labels))
    global_step = tf.train.get_global_step()
    optimizer = tf.train.GradientDescentOptimizer(0.01)
    train = tf.group(optimizer.minimize(loss), tf.assign_add(global_step, 1))
    return tf.contrib.learn.ModelFnOps(mode = mode, predictions = y, loss = loss, train_op = train)
estimator = tf.contrib.learn.Estimator(model_fn = model)
x = np.array([1., 2., 3., 4.])
y = np.array([0., -1., -2., -3.])
input_fn = tf.contrib.learn.io.numpy_input_fn({"x": x}, y, 4, num_epochs = 1000)
estimator.fit(input_fn = input_fn, steps = 1000)
print(estimator.evaluate(input_fn = input_fn, steps = 10))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_keep_checkpoint_max': 5, '_save_summary_steps': 100, '_is_chief': True, '_num_ps_replicas': 0, '_evaluation_master': '', '_save_checkpoints_secs': 600, '_environment': 'local', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_master': '', '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f25869f5ef0>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_tf_random_seed': None}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpvfxmccnn/model.ckpt.
INFO:tensorflow:loss = 5.88955272672, step = 1
INFO:tensorflow:global_step/sec: 1085.82
INFO:tensorflow:loss = 0.00666280672168, step = 101
INFO:tensorflow:global_step/sec: 1010.83
INFO:tensorflow:loss = 0.000364787346446, step = 201
INFO:tensorflow:global_step/sec: 1108.06
INFO:tensorflow:loss = 8.17048081548e-06, s